http://www.hankcs.com/ml/em-algorithm-and-its-generalization.html

## 建立数据集

针对这个问题，首先采集数据，用1表示H（正面），0表示T（反面）：

In [2]:
 import numpy as np
# 硬币投掷结果观测序列
observations = np.array([[1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
                         [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
                         [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
                         [1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0, 1, 1, 1, 0, 1]])

## 初始化

选定初值，比如

## 第一个迭代的E步

抛硬币是一个二项分布，可以用scipy中的binom来计算。对于第一行数据，正反面各有5次，所以：

import scipy.stats as stats
coin_A_pmf_observation_1 = stats.binom.pmf(5,10,0.6)
print(coin_A_pmf_observation_1)

类似地，可以计算第一行数据由B生成的概率：

In [7]:
coin_B_pmf_observation_1 = stats.binom.pmf(5,10,0.5)
print(coin_B_pmf_observation_1)

0.24609375000000025


将两个概率正规化，得到数据来自硬币A的概率：

In [8]:
normalized_coin_A_pmf_observation_1 = coin_A_pmf_observation_1/(coin_A_pmf_observation_1+coin_B_pmf_observation_1)
print ("%0.2f" %normalized_coin_A_pmf_observation_1)

0.45


这个值类似于三硬币模型中的μ，只不过多了一个下标，代表是第几行数据（数据集由5行构成）。

同理，可以算出剩下的4行数据的μ。

有了μ，就可以估计数据中AB分别产生正反面的次数了。

μ代表数据来自硬币A的概率的估计，将它乘上正面的总数，得到正面来自硬币A的总数，同理有反面，同理有B的正反面。

# 更新在当前参数下A、B硬币产生的正反面次数
counts['A']['H'] += weight_A * num_heads

counts['A']['T'] += weight_A * num_tails

counts['B']['H'] += weight_B * num_heads

counts['B']['T'] += weight_B * num_tails

## 第一个迭代的M步

当前模型参数下，AB分别产生正反面的次数估计出来了，就可以计算新的模型参数了：

new_theta_A = counts['A']['H'] / (counts['A']['H'] + counts['A']['T'])

new_theta_B = counts['B']['H'] / (counts['B']['H'] + counts['B']['T'])

## 与论文是一致的，于是就可以整理一下，给出EM算法单个迭代的代码：

In [14]:
def em_single(priors, observations):
    """
    EM算法单次迭代
    Arguments
    ---------
    priors : [theta_A, theta_B]
    observations : [m X n matrix]
 
    Returns
    --------
    new_priors: [new_theta_A, new_theta_B]
    :param priors:
    :param observations:
    :return:
    """
    counts = {'A': {'H': 0, 'T': 0}, 'B': {'H': 0, 'T': 0}}
    theta_A = priors[0]
    theta_B = priors[1]
    # E step
    for observation in observations:
        len_observation = len(observation)
        num_heads = observation.sum()
        num_tails = len_observation - num_heads
        contribution_A = stats.binom.pmf(num_heads, len_observation, theta_A)
        contribution_B = stats.binom.pmf(num_heads, len_observation, theta_B)   # 两个二项分布
        weight_A = contribution_A / (contribution_A + contribution_B)
        weight_B = contribution_B / (contribution_A + contribution_B)
        # 更新在当前参数下A、B硬币产生的正反面次数
        counts['A']['H'] += weight_A * num_heads
        counts['A']['T'] += weight_A * num_tails
        counts['B']['H'] += weight_B * num_heads
        counts['B']['T'] += weight_B * num_tails
    # M step
    new_theta_A = counts['A']['H'] / (counts['A']['H'] + counts['A']['T'])
    new_theta_B = counts['B']['H'] / (counts['B']['H'] + counts['B']['T'])
    return [new_theta_A, new_theta_B]

## EM算法主循环

给定循环的两个终止条件：模型参数变化小于阈值；循环达到最大次数，就可以写出EM算法的主循环了：

In [15]:
def em(observations, prior, tol=1e-6, iterations=10000):
    """
    EM算法
    :param observations: 观测数据
    :param prior: 模型初值
    :param tol: 迭代结束阈值
    :param iterations: 最大迭代次数
    :return: 局部最优的模型参数
    """
    import math
    iteration = 0
    while iteration < iterations:
        new_prior = em_single(prior, observations)
        delta_change = np.abs(prior[0] - new_prior[0])
        if delta_change < tol:
            break
        else:
            prior = new_prior
            iteration += 1
    return [new_prior, iteration]

## 调用

给定数据集和初值，就可以调用EM算法了：

In [17]:
print (em(observations, [0.6, 0.5]))

[[0.7967887593831098, 0.5195839356752803], 14]


与论文中的结果是一致的（我们多迭代了4次，毕竟我们不清楚论文作者设置的终止条件）：

我们可以改变初值，试验初值对EM算法的影响。

In [18]:
em(observations, [0.5,0.6])

[[0.5195834506301285, 0.7967889544439393], 15]

看来EM算法还是很健壮的

如果把初值设为相等会怎样？

调用

In [19]:
em(observations, [0.3,0.3])

[[0.66, 0.66], 1]

这显然是不是个好主意，再试试很极端的情况：

In [20]:
em(observations, [0.9999,0.00000001])

[[0.7967885050458194, 0.5195823568654446], 13]

可见EM算法仍然很聪明。